# BETH Dataset Playground

In [9]:
import pandas as pd
import numpy as np
import scipy as sp
from glob import glob
import networkx as nx
import ipaddress
from scipy import sparse

# Initial Explore
## Load Data

In [10]:
# dns_data_files = glob("*-dns.csv")
# dns_data_files

In [11]:
# feb_process_data_files = glob("*_data.csv")
# feb_process_data_files

### Loading DNS Data

In [12]:
# dns_all = []
# for data in dns_data_files:
#     print(data)
#     df = pd.read_csv(data,parse_dates=True, squeeze=True)
#     dns_all.append(df)
# dns_df = pd.concat(dns_all, ignore_index=True)

In [13]:
# dns_df.head(20)

In [14]:
# dns_df.info()

In [15]:
# dns_df.value_counts('Timestamp')

In [16]:
# duplicateRows = dns_df[dns_df.duplicated()]
# print("Duplicate Rows except first occurrence based on all columns are :")
# print(len(duplicateRows))

In [17]:
# dns_df = dns_df.drop_duplicates()
# dns_df.info()

### Load Process Data

In [18]:
# df_process_feb = []
# for data in feb_process_data_files:
#     df = pd.read_csv(data,parse_dates=True, squeeze=True)
#     df_process_feb.append(df)
# feb_process_df = pd.concat(df_process_feb, ignore_index=True)

In [19]:
# feb_process_df

In [20]:
# feb_process_df.describe()

In [21]:
# feb_process_df.info()

In [22]:
# duplicateRows = feb_process_df[process_df.duplicated()]
# print("Duplicate Rows except first occurrence based on all columns are :")
# print(len(duplicateRows))

In [23]:
# duplicatedTuples = feb_process_df.groupby(['processId','parentProcessId', 'hostName', 'eventName']).size().reset_index(name='Count').sort_values('Count')
# duplicatedTuples

#### Subset Data From Host 10-100-1-186

In [24]:
one86_process_df = pd.read_csv("BETH_data_analysis-main\labelled_2021may-ip-10-100-1-186.csv")

In [25]:
##Separate process nodes from event nodes
one86_procs = one86_process_df[['processId', 'parentProcessId', 'processName', 'userId','timestamp', 'sus', 'evil']].rename(columns={'processName':'name'})
one86_events = one86_process_df[['eventId', 'processId', 'eventName', 'timestamp', 'returnValue','sus', 'evil']].rename(columns={'processId': 'parentProcessId', 'eventName':'name'})

In [26]:
##Remove duplicates
one86_procs = one86_procs.drop_duplicates(keep='first')
one86_events = one86_events.drop_duplicates(keep='first')


In [27]:
# one86_procs.value_counts('processId')
one86_procs[one86_procs['processId'] == 7324]#.value_counts('evil')
#Same process has multiple names and multiple timestamps, both 0/1 sus labels


,processId,parentProcessId,name,userId,timestamp,sus,evil
2799,7324,1,(time-dir),0,143.928992,0,0
2800,7324,1,(time-dir),0,143.929058,0,0
2801,7324,1,(time-dir),0,143.929081,0,0
2802,7324,1,(time-dir),0,143.929102,0,0
2803,7324,1,(time-dir),0,143.929122,0,0
...,...,...,...,...,...,...,...
3333,7324,1,systemd-user-ru,0,143.978131,0,0
3334,7324,1,systemd-user-ru,0,143.978151,0,0
3335,7324,1,systemd-user-ru,0,143.978171,0,0
3336,7324,1,systemd-user-ru,0,143.978194,0,0


In [28]:
one86_events[one86_events['sus'] == 1]

,eventId,parentProcessId,name,timestamp,returnValue,sus,evil
67,3,7285,close,125.595818,0,1,0
68,3,7285,close,125.595860,0,1,0
69,1010,7285,sched_process_exit,125.595990,0,1,0
70,1010,1368,sched_process_exit,125.596139,0,1,0
71,3,1355,close,125.597087,0,1,0
...,...,...,...,...,...,...,...
712057,257,159,openat,16022.604230,33,1,0
712067,1005,159,security_file_open,16022.642637,0,1,0
712068,257,159,openat,16022.642668,34,1,0
712079,1005,159,security_file_open,16022.682575,0,1,0


In [29]:
# one86_events.value_counts('eventId')
# one86_events[(one86_events['eventId'] == 3)]
one86_events[(one86_events['eventId'] == 3) & (one86_events['parentProcessId'] == 380)]


,eventId,parentProcessId,name,timestamp,returnValue,sus,evil
10,3,380,close,124.954321,0,0,0
3351,3,380,close,173.977672,0,0,0
4561,3,380,close,224.281660,0,0,0
5747,3,380,close,281.625738,0,0,0
7197,3,380,close,373.785678,0,0,0
...,...,...,...,...,...,...,...
672605,3,380,close,15662.873652,0,0,0
675877,3,380,close,15739.161703,0,0,0
678377,3,380,close,15809.561756,0,0,0
682341,3,380,close,15872.793708,0,0,0


In [30]:
#same event occurs many times, even for same process, but at different times-- are they ALL sus or ALL evil?
groups = one86_events.groupby(by=['eventId', 'parentProcessId']).agg({'name':'count','sus':'sum', 'evil': 'sum'}).rename(columns={'name':'count'})
groups[(groups['count']!=groups['sus']) & (groups['sus']>0)]
#NO

count  sus  evil
eventId parentProcessId                  
87      510                  7    1     0
257     159              27066    3     0
        7324                71   11     0
1005    159              10609    3     0
        7324                55   10     0
1006    510                  7    1     0

In [31]:
##concatenate parentProcessId to eventId to differentiate
one86_events['eventId'] =  (one86_events['eventId'].astype(str) + '.' + one86_events['parentProcessId'].astype(str)).astype(float)

In [32]:
##Propagate sus/evil, Remove conflicting names, keep first process timestamp
one86_procs = one86_procs.groupby('processId').agg({'parentProcessId':'min', 'name':'first', 'userId':'max', 'timestamp':'min', 'sus':'max', 'evil': 'max'})
one86_events = one86_events.groupby('eventId').agg({'parentProcessId':'first', 'name':'first', 'timestamp':'first', 'returnValue':'max', 'sus': 'max', 'evil': 'max'})

In [33]:
##One-hot return value - negative: 1-0 / zero: 0-0 / positive: 0-1
one86_events['returnNegative'] = np.where(one86_events['returnValue'] < 0, 1, 0)
one86_events['returnPositive'] = np.where(one86_events['returnValue'] > 0, 1, 0)

##Binarize userId - 0-1000: 0, 1000+: 1
one86_procs['userType'] = np.where(one86_procs['userId'] >= 1000, 1, 0)


In [34]:
##Feature for type of node
one86_procs['hostType'] = 0
one86_procs['processType'] = 1
one86_events['hostType'] = 0
one86_events['processType'] = 0


In [35]:
one86_procs

,parentProcessId,name,userId,timestamp,sus,evil,userType,hostType,processType
processId,,,,,,,,,
1,0,systemd,0,124.953424,0,0,0,0,1
5,2,kworker/dying,0,709.656880,0,0,0,0,1
7,2,kworker/dying,0,345.124745,0,0,0,0,1
8,2,kworker/dying,0,2823.192768,0,0,0,0,1
78,2,kworker/dying,0,318.744790,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
8903,7096,sshd,0,15997.377169,0,0,0,0,1
8904,8903,sshd,109,15997.698131,0,0,0,0,1
8906,1280,amazon-ssm-agen,0,16012.896073,0,0,0,0,1


In [36]:
one86_events

,parentProcessId,name,timestamp,returnValue,sus,evil,returnNegative,returnPositive,hostType,processType
eventId,,,,,,,,,,
3.1000,1,close,124.953618,0,0,0,0,0,0,0
3.1113,1113,close,133.660742,0,1,0,0,0,0,0
3.1170,1170,close,133.665825,0,0,0,0,0,0,0
3.1182,1182,close,143.386891,0,0,0,0,0,0,0
3.1279,1279,close,133.661689,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1010.8904,8904,sched_process_exit,16001.627323,0,0,0,0,0,0,0
1010.8906,8906,sched_process_exit,16013.161577,0,0,0,0,0,0,0
1010.8908,8908,sched_process_exit,16021.341714,0,0,0,0,0,0,0


## Graphs

### Process Graph

#### Subset Data Graph

In [37]:
one86_process_df = one86_procs.reset_index().rename(columns={'processId':'id'}).append(one86_events.reset_index().rename(columns={'eventId':'id'}))

In [38]:
one86_process_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22121 entries, 0 to 20615
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               22121 non-null  float64
 1   parentProcessId  22121 non-null  int64  
 2   name             22121 non-null  object 
 3   userId           1505 non-null   float64
 4   timestamp        22121 non-null  float64
 5   sus              22121 non-null  int64  
 6   evil             22121 non-null  int64  
 7   userType         1505 non-null   float64
 8   hostType         22121 non-null  int64  
 9   processType      22121 non-null  int64  
 10  returnValue      20616 non-null  float64
 11  returnNegative   20616 non-null  float64
 12  returnPositive   20616 non-null  float64
dtypes: float64(7), int64(5), object(1)
memory usage: 2.4+ MB


In [39]:
one86ProcessGraph = nx.from_pandas_edgelist(one86_process_df, source="id", target="parentProcessId", create_using=nx.MultiDiGraph())
## creating 22129 nodes

In [40]:
print("Graph analysis\n")
print("Multi-edge directed Graph\n")
print("Number of nodes: %s\t" % len(one86ProcessGraph.nodes))
print("Number of edges: %s\n" % len(one86ProcessGraph.edges))
print("Graph density: %s\n" % nx.density(one86ProcessGraph))
print("Graph is directed: %s\n" % one86ProcessGraph.is_directed())
print("Graph is weighted: %s\n" % nx.is_weighted(one86ProcessGraph))

Graph analysis

Multi-edge directed Graph

Number of nodes: 22129	
Number of edges: 22121

Graph density: 4.517527492037339e-05

Graph is directed: True

Graph is weighted: False



In [41]:
one86_attributes = one86_process_df[['id','userId','timestamp', 'returnNegative', 'returnPositive','sus','evil','userType','hostType','processType']].set_index('id').to_dict('index')

In [42]:
nx.set_node_attributes(one86ProcessGraph, one86_attributes)

### DNS Graph

# To DGI Format

### DNS Data to DGI format

### Process Data to DGI format

#### Subset Data to DGI Format

In [43]:
one86_process_df

,id,parentProcessId,name,userId,timestamp,sus,evil,userType,hostType,processType,returnValue,returnNegative,returnPositive
0,1.0000,0,systemd,0.0,124.953424,0,0,0.0,0,1,NaN,NaN,NaN
1,5.0000,2,kworker/dying,0.0,709.656880,0,0,0.0,0,1,NaN,NaN,NaN
2,7.0000,2,kworker/dying,0.0,345.124745,0,0,0.0,0,1,NaN,NaN,NaN
3,8.0000,2,kworker/dying,0.0,2823.192768,0,0,0.0,0,1,NaN,NaN,NaN
4,78.0000,2,kworker/dying,0.0,318.744790,0,0,0.0,0,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20611,1010.8904,8904,sched_process_exit,NaN,16001.627323,0,0,NaN,0,0,0.0,0.0,0.0
20612,1010.8906,8906,sched_process_exit,NaN,16013.161577,0,0,NaN,0,0,0.0,0.0,0.0
20613,1010.8908,8908,sched_process_exit,NaN,16021.341714,0,0,NaN,0,0,0.0,0.0,0.0
20614,1010.8909,8909,sched_process_exit,NaN,16021.340781,0,0,NaN,0,0,0.0,0.0,0.0


In [44]:
adj = nx.adjacency_matrix(one86ProcessGraph, nodelist=one86_process_df['id']) ##scipy.sparse.csr.csr_matrix
## correct order?
## 8 extra nodes! without nodelist

In [45]:
# features = nx.get_node_attributes(one86ProcessGraph, 'name') #dict
features = sparse.lil_matrix(np.matrix(one86_process_df[['userId','timestamp', 'returnNegative', 'returnPositive','userType','hostType','processType']])) ##scipy.sparse.lil.lil_matrix
##does order match adj???

In [46]:
labels = np.asarray(one86_process_df[['sus','evil',]])##numpy.ndarray

In [47]:
idx_train = range(1000)##range
idx_val = range(1000, 1500)##range
idx_test = [range(1500,2000)]##list
